In [ ]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

# from Kernel import MixtureViaSumAndProduct, CategoryOverlapKernel
from InitialData_Gen import initialize
from AcquisitionFunctions import EI, PI, UCB, AcquisitionOnSubspace
# from SamplingCategorical import compute_prob_dist_and_draw_hts
from UpdateCategoricalWeight import compute_reward_for_all_cat_variable, update_weights_for_all_cat_var
from optimization import sample_then_minimize

from AskTell import ask_tell

from scipy.optimize import minimize

from typing import Union, Tuple
from paramz.transformations import Logexp

1. C - List of number of categroies corresponding to each categorical variable
2. Nc - Number of categorical variables
3. Nx - Continuous variables
4. h - categorical
5. x - continuous
6. z - [h, x]

7. nDim - Nc + Nx
8. bounds - Lower and Upper bounds of both Catergorical variables and continuous variables

9. n_iter - Number of iterations to run the algorithm
10. initN - Number of intial data points
11. batch_size (b) - Number of experiments to be generated in each iteration

12. acq_type - Acquisition Type ('EI', 'LCB', 'ThompsonSampling')

### 1. Load Model background from the pickle file

In [ ]:
import sys
myString = sys.path[0]
split_list = myString.split("/")
root = ''
for l in split_list[1:-2]:
    root = root +'/'+ l
    
main_file_path = root + '/HSA_TL/'
carbon_source_fiepath = root + '/'

In [ ]:
Stock_solid = pd.read_excel(carbon_source_fiepath + 'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_fiepath + 'CarbonSourceInfo.xlsx', 'Stocks_liquid')

Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

Glu_Ub = 50 #mM
Tween_Ub = 1# 1%
pH_Ub = 6.5# 

Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()

Glu_Stock_Conc = 250 #mM
Tween_Stock_Conc = 50# %
pH_Stock_Conc = 1# 

### 2. Read Experimental condition and results 

In [ ]:
#Load a titer quantification file and create the result array
alt_path = root
AllData = pd.read_csv(alt_path +'AllMolecule/HSA_CoCaBO_Prod_AllData.csv')
result = AllData['SP'].values.reshape(-1,1)

data = AllData.iloc[:,1:5].values

In [ ]:
Carbon_mL = []
OG_Gly_mL = []
Met_mL = []

for i in range(data.shape[0]):
    if int(data[i,0]) !=19:
        temp_factor = Carbon_Ub[int(data[i,0])]
    else:
        temp_factor = Carbon_Ub[0]
    temp = data[i,1]/(temp_factor)
    Carbon_mL.append(temp)
    OG_Gly_mL.append(data[i,2]/OG_Gly_Ub * 1000/100)
    Met_mL.append(data[i,3]/Met_Ub*1000/100)
    
data_sc = np.concatenate((data[:,0:1],np.array(Carbon_mL).reshape(-1,1),
                          np.array(OG_Gly_mL).reshape(-1,1), np.array(Met_mL).reshape(-1,1)), axis = 1)

data_sc = np.concatenate((data_sc, 
                          1e-5 * np.ones((AllData.shape[0],4)), 
                          0.999 * np.ones((AllData.shape[0],1))), axis = 1)



In [ ]:
#Load a titer quantification file and create the result array
N_round = 1
Design = {}
Result_df = {}
for nr in range(N_round):
    file_name = main_file_path + 'Codes/Round' + str(nr) + '/Reconstructed_Round' + str(nr) + '.csv'
    Design[nr] = pd.read_csv(file_name)
    Column_Names = pd.read_csv(file_name).columns
    file_name_res = main_file_path + 'Exp/Round' + str(nr) + '/Round' + str(nr) + '_Result_Summary_final.csv'
    Result_df[nr] = pd.read_csv(file_name_res)
    fac = int(Result_df[nr].shape[0]/Design[nr].shape[0])
    temp = np.repeat(Design[nr].iloc[:,1:].values,fac, axis = 0)
#     print(temp)
# #     for j in range(Design[nr].shape[0]):
#         temp = np.repeat(Design[nr].iloc[:,1:].values,2, axis = 0)
    data_sc = np.concatenate((data_sc, temp), axis = 0)
        
    result = np.concatenate((result, Result_df[nr]['Specific Productivity'].iloc[:-1,].values.reshape(-1,1)), axis = 0)
    
mu_y = np.mean(result, 0)
std_y = np.std(result, 0)

mu_x = np.mean(data_sc, 0)
std_x = np.std(data_sc, 0)

data_norm = (data_sc - mu_x)/std_x
data_norm[:,0] = data_sc[:,0]

result_norm = (result - mu_y)/std_y

In [ ]:
Nc = 1
C_list = [19] #

assert Nc == len(C_list)

Nx = 8 #3+5
nDim = Nc + Nx
Niter = 1
batch_size = 10
initN = data.shape[0]

bounds = [{'name': 'Carbon_Type', 'type': 'categorical', 'domain': np.arange(0, 19)}, #Carbon Source type
          {'name': 'Conc_Carbon', 'type': 'continuous', 'domain': (0, 1)},
            {'name': 'Gly_OG', 'type': 'continuous', 'domain': (0, 1)},
            {'name': 'Met_Prod', 'type': 'continuous', 'domain': (0, 1)},
         {'name': 'Glu_OG', 'type': 'continuous', 'domain': (0, 1)},
         {'name': 'Tween_OG', 'type': 'continuous', 'domain': (0, 1)},
         {'name': 'Glu_Prod', 'type': 'continuous', 'domain': (0, 1)},
         {'name': 'Tween_Prod', 'type': 'continuous', 'domain': (0, 1)},
         {'name': 'pH', 'type': 'continuous', 'domain': (5.75/6.5, 6.5/6.5)}]


# load data from pkl file
background_file = main_file_path +  "Codes/Round0/5_ModelBackground_HSA_Prod.pkl"
with open(background_file, "rb") as fp:
    ModelBackground_0 = pickle.load(fp)
    
ModelBackground_0['data_param']['Meas_Noise'] = 0.034
ModelBackground_0['data_param']['tarde_off'] = 3
ModelBackground_0['data_param']['Nx'] = Nx
ModelBackground_0['data_param']['nDim'] = nDim
ModelBackground_0['data_param']['bounds'] = bounds 

In [ ]:
ModelBackground_0['data_param']

### 3. First iteration to ask the algorithm

In [ ]:
#Update Weights from the CoCaBO_prod with results from run 0 (Only done here, since round 0 was generated with old code bundle
#New code bundle already has this incorporated). From next round, this can be skipped
batch_size = 11
ht_next_list_array = np.zeros((batch_size,1))
ht_next_list_array[:,0:1] = np.atleast_2d(np.array(ModelBackground_0['Categorical_dist_param']['ht_batch_list']))
ht_list_rewards = compute_reward_for_all_cat_variable(ht_next_list_array, C_list,
                                                      data,result_norm, batch_size) # np.divide(result,np.max(result))

Wc_list = update_weights_for_all_cat_var(C_list, 
                ht_list_rewards, ht_next_list_array ,
                 ModelBackground_0['Categorical_dist_param']['Wc_list'], ModelBackground_0['gamma_list'],
                ModelBackground_0['Categorical_dist_param']['probabilityDistribution_list'],
                batch_size, [])

In [ ]:
batch_size = 10

random.seed(0)

z_next, Categorical_dist_param, gp_actual = ask_tell(data_sc, result, ModelBackground_0['data_param'], 
                                          'RBF', 'constant_liar',  batch_size, 
                                           Wc_list,  
                                          ModelBackground_0['gamma_list'])

In [ ]:
pd.DataFrame(z_next).to_csv('./Round1/1_ExperimentalDesign.csv')

### 4. Store the Model background parameters

In [ ]:
ModelBackground_1 = {}
ModelBackground_1 = {'gamma_list': ModelBackground_0['gamma_list'],  'budget': ModelBackground_0['budget'],
                 'bestUpperBoundEstimate': ModelBackground_0['bestUpperBoundEstimate'], 
                    'data_param': ModelBackground_0['data_param'], 
                   'Categorical_dist_param': Categorical_dist_param}

import pickle
with open('./Round1/1_ModelBackground.pkl', 'wb') as output:
    # Pickle dictionary using protocol 0.
    pickle.dump(ModelBackground_1, output)

### 5. Conversion to actual experimental execution

In [ ]:
Selected_Carbon = []
Carbon_mL = []
OG_Gly_mL = []
Met_mL = []
Glu_OG_mL = []
Tween_OG_mL = []
Glu_Prod_mL = []
Tween_Prod_mL = []
pH_mL = []

for i in range(batch_size):
    Selected_Carbon.append(Carbon_Names[int(z_next[i,0])])
    temp_factor = Carbon_Ub[int(z_next[i,0])]/ Stock_Conc[int(z_next[i,0])]
    temp = z_next[i,1] * temp_factor * 3
    # [g/L]/[g/mL] * [mL] --> g * mL * mL/[L * g] --> 10^-6 L --> uL 
    Carbon_mL.append(temp)
    # [mL]*[mL/L] --> 10^-6 L --> 1 uL 
    OG_Gly_mL.append(z_next[i,2] * OG_Gly_Ub * 3/OG_Stock_Conc)
    Met_mL.append(z_next[i,3] * Met_Ub * 3)
    
    Glu_OG_mL.append(z_next[i,4] * Glu_Ub * 3 * 1000/Glu_Stock_Conc) # uL
    Tween_OG_mL.append(z_next[i,5] * Tween_Ub * 3 * 1000/Tween_Stock_Conc) #uL
    Glu_Prod_mL.append(z_next[i,6] * Glu_Ub * 3 * 1000/Glu_Stock_Conc) # uL
    Tween_Prod_mL.append(z_next[i,7] * Tween_Ub * 3 * 1000/Tween_Stock_Conc) #uL
    pH_mL.append(z_next[i,8] * pH_Ub ) #* 3/pH_Stock_Conc
    


Experiment_1_3mL = {'Carbon_Type': Selected_Carbon,
               'Conc_Carbon [uL]': Carbon_mL,
               'Gly_OG [uL]': OG_Gly_mL,
               'Met_Prod [uL]': Met_mL,
                'Glu_OG [uL]' :Glu_OG_mL,
                'Tween_OG [uL]':Tween_OG_mL,
                'Glu_Prod [uL]': Glu_Prod_mL ,
                'Tween_Prod [uL]': Tween_Prod_mL ,  
                  'pH': pH_mL }

pd.DataFrame(Experiment_1_3mL).to_csv('./Round1/1_ExperimentPlan_mLValue_3mL.csv')

In [ ]:
plt.scatter(data_sc[:,0], data_sc[:,1])
plt.scatter(z_next[:,0],z_next[:,1])

In [ ]:
plt.scatter(data_sc[:,2], data_sc[:,3])
plt.scatter(z_next[:,2],z_next[:,3])

In [ ]:
plt.scatter(data_sc[:,4], data_sc[:,5])
plt.scatter(z_next[:,4],z_next[:,5])

In [ ]:
plt.scatter(data_sc[:,5], data_sc[:,6])
plt.scatter(z_next[:,5],z_next[:,6])

In [ ]:
plt.scatter(data_sc[:,7], data_sc[:,8])
plt.scatter(z_next[:,7],z_next[:,8])

In [ ]:
pd.DataFrame(Experiment_1_3mL).to_csv('./Round1/Actual_Round1_3mL.csv')